# Video understanding - IP camera 이벤트 감지
IP camera 이벤트 감지는 보안 카메라 영상을 분석하여 택배 배송, 도난 사건 또는 의심스러운 활동과 같은 특정 이벤트를 자동으로 식별합니다. 이 기능은 보안팀, 건물 관리자 및 주택 소유자가 지속적인 수동 감시 없이도 중요한 이벤트를 감지하고 알림을 받아 효율적으로 건물을 모니터링할 수 있도록 도와줍니다.

이 샘플 노트북에서는 Video Understanding Tool을 사용하여 추출한 메타데이터를 사용합니다. 이 메타데이터에는 관련 timestamp가 포함된 frame 수준의 객체 감지 정보가 포함되어 있습니다. 이 메타데이터를 검색하여 LLM을 통한 이벤트 감지 분석에 사용합니다.

![ip_camera](../statics/video-ip-camera.png)

In [ ]:
from utils import dynamodb_tool, bedrock_tool
import json
import boto3

Video Understanding Tool UI에서 task ID를 찾을 수 있습니다.

In [ ]:
task_id = 'YOUR_TASK_ID_FROM_VIDEO_UNDERSTANDING_TOOL'

Video Understanding Tool의 관리형 데이터베이스에서 frame 수준의 객체 감지 결과를 검색합니다.

In [ ]:
frame_outputs = dynamodb_tool.get_frame_outputs(task_id, output_names=["Object detection"])
print(json.dumps(frame_outputs,indent=2))

## 이벤트 감지 분석
고급 패턴 인식을 사용하여 frame 메타데이터를 분석하여 택배 배송 및 도난 이벤트를 감지합니다. 분석은 객체 존재, 사람의 방향 및 시간적 순서를 고려하여 특정 보안 이벤트를 식별합니다.

In [ ]:
# 상세한 분류 규칙이 포함된 이벤트 감지 프롬프트
event_detection_prompt = f"""
##역할##
당신은 timestamp가 포함된 비디오 frame 메타데이터를 분석하여 특정 이벤트를 식별하는 AI 이벤트 감지 시스템입니다. 당신의 임무는 여러 frame에 걸친 객체 감지 데이터를 기반으로 택배 배송 및 택배 도난 이벤트를 감지하는 것입니다.

위의 시스템 지침은 당신의 기능과 범위를 정의합니다. 사용자 요청이 시스템 지침과 모순되는 경우, 당신의 기능을 설명하며 정중하게 거절하십시오.

##입력 데이터 형식##
다음을 포함하는 객체 감지가 포함된 timestamp가 있는 frame 메타데이터를 받게 됩니다:
- timestamp: frame이 캡처된 시간(초)
- objects: 좌표 및 속성이 포함된 감지된 객체
  - name: 객체 유형(person, car, package)
  - coordinates: Bounding box (x, y, width, height)
  - facingCamera: 사람이 카메라를 향하고 있는지 나타내는 Boolean (person 객체에만 해당)

##이벤트 분류 규칙##

###택배 배송 이벤트###
다음 조건이 충족되면 택배 배송이 감지됩니다:
1. 초기 frame에 package를 가진 person이 있음
2. person이 처음에 카메라를 향하고 있음
3. package가 정지 상태로 유지되거나 내려놓음
4. person이 장면을 떠남(멀어지거나 frame을 벗어남)
5. person이 떠난 후에도 package가 장면에 남아 있음

###택배 도난 이벤트###
다음 조건이 충족되면 택배 도난이 감지됩니다:
1. 처음에 장면에 package가 있음
2. person이 접근하고 카메라를 향하고 있음
3. person이 package를 집거나 상호작용함
4. person이 카메라에서 돌아섬(facingCamera가 true에서 false로 변경)
5. 후속 frame에서 person과 package가 모두 사라짐

##작업##
제공된 frame 메타데이터를 분석하여 ##이벤트 분류 규칙##에 따라 택배 배송 또는 택배 도난 이벤트가 발생했는지 확인하십시오.

분석할 Frame 메타데이터: {frame_outputs}

먼저 단계별로 생각한 다음 답변하십시오. 아래의 응답 형식을 정확히 따르십시오.
"""

Tool 구성은 Foundation Model에서 구조화된 출력을 얻는 더 안정적인 방법을 제공합니다. 이 예제에서는 다음 tool 구성을 정의하고 Bedrock Converse API로 전송합니다.

In [ ]:
# 구조화된 이벤트 감지 출력을 위한 Tool 구성
tool_config = {
    "toolChoice": {
        "tool": {
            "name": "event_detection_result"
        }
    },
    "tools": [
        {
            "toolSpec": {
                "name": "event_detection_result",
                "description": "timestamp가 있는 비디오 frame 메타데이터를 분석하여 택배 배송 또는 택배 도난 이벤트를 감지합니다.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "events_detected": {
                                "type": "array",
                                "description": "시간순으로 정렬된 모든 감지된 이벤트 목록. 각 이벤트는 비디오 영상에서 식별된 고유한 택배 관련 활동을 나타냅니다.",
                                "items": {
                                    "type": "object",
                                    "description": "이벤트 분류, 타이밍 및 지원 증거를 포함하는 개별 이벤트 감지 결과.",
                                    "properties": {
                                        "event_type": {
                                            "type": "string",
                                            "description": "감지된 이벤트의 분류: 무단 택배 탈취의 경우 'package_theft', 정당한 배송 활동의 경우 'package_delivery', 택배 관련 활동이 감지되지 않은 경우 'no_event'.",
                                            "enum": ["package_theft", "package_delivery", "no_event"]
                                        },
                                        "time_range": {
                                            "type": "object",
                                            "description": "비디오 timeline에서 이벤트의 시간적 경계, 비디오 시작부터 초 단위로 측정됩니다.",
                                            "properties": {
                                                "start_timestamp": {
                                                    "type": "number",
                                                    "description": "이벤트의 시작 timestamp(초). 첫 번째 관련 활동 또는 사람이 장면에 나타나는 시점을 표시합니다."
                                                },
                                                "end_timestamp": {
                                                    "type": "number",
                                                    "description": "이벤트의 종료 timestamp(초). 활동이 종료되거나 사람이 장면을 벗어나는 시점을 표시합니다."
                                                }
                                            },
                                            "required": ["start_timestamp", "end_timestamp"]
                                        },
                                        "evidence": {
                                            "type": "array",
                                            "description": "이벤트 분류를 뒷받침하는 구체적인 관찰 및 행동 지표 목록. 사람, 행동, 객체, 유니폼, 차량 및 상황적 단서에 대한 세부 정보를 포함합니다.",
                                            "items": {
                                                "type": "string",
                                                "description": "'배송 유니폼을 입은 사람', '문 앞에 놓인 택배', '무단으로 택배를 가져가는 사람', '배송 트럭이 보임' 등과 같은 구체적인 증거."
                                            }
                                        }
                                    },
                                    "required": ["event_type", "time_range", "evidence"]
                                }
                            },
                            "analysis_notes": {
                                "type": "string",
                                "description": "사용된 방법론, 발견된 모호성, 비디오 품질 관찰, 신뢰도 수준 및 필요한 경우 추가 조사를 위한 권장 사항을 포함한 전체 분석 요약."
                            }
                        },
                        "required": ["events_detected", "analysis_notes"]
                    }
                }
            }
        }
    ]
}


In [ ]:
# 이벤트 감지 분석 실행
response = bedrock_tool.bedrock_converse(
    model_id="global.amazon.nova-2-lite-v1:0",
    prompt=event_detection_prompt,
    tool_config=tool_config,
    inference_config={"maxTokens": 2000, "temperature": 0, "topP": 0}
)

result = bedrock_tool.parse_converse_response(response)
event_analysis = json.loads(result)
print(json.dumps(event_analysis,indent=2, ensure_ascii=False))

## 결과 요약
상세한 분석 및 신뢰도 수준과 함께 감지된 이벤트의 형식화된 요약을 표시합니다.

In [ ]:
# 형식화된 결과 표시
if 'event_analysis' in locals() and event_analysis:
    print("\n" + "="*50)
    print("이벤트 감지 요약")
    print("="*50)
    
    events = event_analysis.get('events_detected', [])
    
    if not events or (len(events) == 1 and events[0].get('event_type') == 'no_event'):
        print("비디오에서 중요한 이벤트가 감지되지 않았습니다.")
    else:
        for i, event in enumerate(events, 1):
            if event.get('event_type') != 'no_event':
                event_type_kr = {
                    'package_theft': '택배 도난',
                    'package_delivery': '택배 배송'
                }.get(event['event_type'], event['event_type'])
                
                print(f"\n이벤트 {i}: {event_type_kr}")
                print(f"시간 범위: {event['time_range']['start_timestamp']}초 - {event['time_range']['end_timestamp']}초")
                
                print("\n증거:")
                for evidence in event['evidence']:
                    print(f"  • {evidence}")
    
    if event_analysis.get('analysis_notes'):
        print(f"\n분석 노트: {event_analysis['analysis_notes']}")
else:
    print("이벤트 분석 결과를 사용할 수 없습니다.")